# 02a - Flash Loan Classification
Classify flash loans based on their purpose. Reads in the tx hash from the flash loan event file.

Workflow:
* Input - Takes the existing flash loan events from 01a
* Output - Creates two additional tables
    * tx_info - metadata info
    * logs - all log data related to the transaction

In [174]:
import pandas as pd
from web3 import Web3, HTTPProvider
import ctc
import os
from tqdm.notebook import tqdm

In [126]:
ETH_RPC_URL = os.getenv("WEB3_PROVIDER_URI")
w3 = Web3(HTTPProvider(ETH_RPC_URL))

In [208]:
def get_tx_metadata(tx_info) -> pd.DataFrame:
    '''
    Cleans the transaction metadata from the transaction_receipt API and returns a dataframe row.
    ----------
    
    Params:
        tx_info, dict:
        
    ----------
    Returns:
        dataframe
    
    '''
    return (
        pd.DataFrame(tx_info)
        .head(n=1)
        [['blockHash', 'transactionHash', 'blockNumber', 'contractAddress', 'gasUsed', 'cumulativeGasUsed', 'effectiveGasPrice', 'from', 'to', 'status', 'transactionIndex', 'type']]
        .assign(
                transactionHash = lambda x: x.transactionHash.apply(lambda x: x.hex()),
                blockHash = lambda x: x.blockHash.apply(lambda x: x.hex())
            ) 
    )
    
def get_log_data(tx_info) -> pd.DataFrame:
    '''
    Cleans the log data from the transaction_receipt API and returns a dataframe row.
    ----------
    
    Params:
        tx_info, dict:
        
    ----------
    Returns:
        dataframe
    
    '''
    return (
        pd.DataFrame(tx_info['logs'])
        .drop(columns=['transactionIndex', 'blockHash'])
        .assign(
                data = lambda x: x.data.apply(lambda x: x.hex()),
                transactionHash = lambda x: x.transactionHash.apply(lambda x: x.hex())
        )  
    )

# Ethereum Data

In [209]:
df_ethereum = pd.read_parquet("../data/flash_loans/ethereum_aave_v3_flash_loans.parquet")

In [210]:
log_data = pd.DataFrame()
tx_metadata = pd.DataFrame()

# Iterate through each tx_hash and concat the log and metadata information to seperate dataframes
for tx_hash in tqdm(df_ethereum['evt_tx_hash'].unique()):
    tx_info = dict(w3.eth.get_transaction_receipt(tx_hash))

    log_data = pd.concat([log_data, get_log_data(tx_info)])
    tx_metadata = pd.concat([tx_metadata, get_tx_metadata(tx_info)], ignore_index=True)

  0%|          | 0/8836 [00:00<?, ?it/s]

In [211]:
tx_metadata.to_parquet("../data/tx_data/ethereum_tx_metadata.parquet")

In [212]:
log_data.to_parquet("../data/tx_data/ethereum_tx_logs.parquet")

In [ ]:
address_book = {
    '0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2': 'Aave V3',
    '0x8761e0370f94f68Db8EaA731f4fC581f6AD0Bd68': 'Debt Swapper for Aave V3',
    '0xB748952c7BC638F31775245964707Bcc5DDFabFC': 'Aave Migration Helper',
    '0x8f30ADaA6950b31f675bF8a709Bc23F55aa24735': 'Debt Swapper for Aave V3',
    '0x1809f186D680f239420B56948C58F8DbbCdf1E18': 'Repay for Aave V3',
    '0x872fBcb1B582e8Cd0D0DD4327fBFa0B4C2730995': 'Collateral Swapper for Aave V3',
    '0x9901bAC880cAecAd999e292811DB9C1Db3e86F8a': 'Furucombo',
    '0x036c4567dB47CD840dE37e7e8658b0bB3ae95d40': 'Liquis',
    '0x775C559D9A48cE5a8444C1035C3a8921ab477b8e': 'MEV Bot',
    '0x252025dF8680C275D0bA80D084e5967D8BD26caf': 'DeFi Saver'
}

In [262]:
contract_abi = await ctc.async_get_contract_abi(
        contract_address='0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'
    )

fetching mainnet abi from etherscan: 0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48
etherscan ratelimit hit, sleeping for 4.605483770370483 seconds
fetching mainnet abi from etherscan: 0x43506849d7c04f9138d1a2050bbf3a0c054402dd


In [285]:
contract_abi[21]

{'anonymous': False,
 'inputs': [{'indexed': True,
   'internalType': 'address',
   'name': 'from',
   'type': 'address'},
  {'indexed': True,
   'internalType': 'address',
   'name': 'to',
   'type': 'address'},
  {'indexed': False,
   'internalType': 'uint256',
   'name': 'value',
   'type': 'uint256'}],
 'name': 'Transfer',
 'type': 'event'}

What is contained in the transaction metadata?

In [260]:
example_tx = tx_metadata.iloc[0].copy()
example_tx

blockHash            0x23a0fa510c09244b3c8f08521373e49fc34ac7ed3728...
transactionHash      0x625ed737a7a97875f7707bb76a7f4075b96709c70d79...
blockNumber                                                   18709117
contractAddress                                                   None
gasUsed                                                        1470387
cumulativeGasUsed                                             15525360
effectiveGasPrice                                          40987012669
from                        0x9fBa5259A1a4654212849E87CFdf5B28d5bbcB5e
to                          0xa6a147946FACAc9E0B99824870B36088764f969F
status                                                               1
transactionIndex                                                    91
type                                                                 2
Name: 0, dtype: object

What is contained in the log data?

In [261]:
# Take the first event in the file
example_event = log_data.iloc[0].copy()

# List relevant info
print(f"Transaction hash: {example_event['transactionHash']}")
print(f"Block: {example_event['blockNumber']} at index: {example_event['logIndex']}")
print(f"Address: {example_event['address']}")
print(f"Topics:")
print(f"\tTopic 0: {example_event['topics'][0].hex()}")
print(f"\tTopic 1: {example_event['topics'][1].hex()}")
print(f"\tTopic 2: {example_event['topics'][2].hex()}")
print(f"Data: {example_event['data']}")

Transaction hash: 0x625ed737a7a97875f7707bb76a7f4075b96709c70d79334d42e0e3c4eed9d7f0
Block: 18709117 at index: 300
Address: 0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48
Topics:
	Topic 0: 0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef
	Topic 1: 0x0000000000000000000000009fba5259a1a4654212849e87cfdf5b28d5bbcb5e
	Topic 2: 0x0000000000000000000000003f37c7d8e61c000085aac0515775b06a3412f36b
Data: 0x0000000000000000000000000000000000000000000000000000000d85bd346f


Given ABI information, you can decode this as a USDC Transfer event. Topic 0 contains the Bytes signature (0xddf252ad) which corresponds to the text signature: Transfer(address,address,uint256)
* Sender (Topic 1): 0x9fba5259a1a4654212849e87cfdf5b28d5bbcb5e
* Receiver (Topic 2): 0x3f37c7d8e61c000085aac0515775b06a3412f36b
* Amount (Data): 58078344303 (USDC uses 6 decimals)

In [255]:
58078344303/(10**6)

58078.344303